In [5]:
import pandas as pd
df= pd.read_csv("heart(in).csv")
df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [6]:
df.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [7]:
df.duplicated().sum()

0

In [8]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [9]:
X = df.drop(["HeartDisease"], axis=1)  
y = df["HeartDisease"]

In [10]:
print(X.dtypes)


Age                 int64
Sex                object
ChestPainType      object
RestingBP           int64
Cholesterol         int64
FastingBS           int64
RestingECG         object
MaxHR               int64
ExerciseAngina     object
Oldpeak           float64
ST_Slope           object
dtype: object


In [11]:
X_encoded = pd.get_dummies(X)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [13]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
model = Sequential()
model.add(Dense(units=64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=1, activation='sigmoid'))

C:\Users\sania\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=7,
    restore_best_weights=True,
    verbose=1
)

In [15]:
checkpoint = ModelCheckpoint(
    'car_model.keras',
    monitor = 'val_loss',
    save_best_only=True,
    verbose=1
)

In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │           1,344 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,457 (13.50 KB)

 Trainable params: 3,457 (13.50 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
history = model.fit(X_train, y_train, batch_size=20, epochs=15, validation_split=0.3, callbacks=[early_stop, checkpoint], verbose=1)

Epoch 1/15
23/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5862 - loss: 0.6817
Epoch 1: val_loss improved from inf to 0.45764, saving model to car_model.keras
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 41ms/step - accuracy: 0.5999 - loss: 0.6692 - val_accuracy: 0.8326 - val_loss: 0.4576
Epoch 2/15
23/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7957 - loss: 0.4725
Epoch 2: val_loss improved from 0.45764 to 0.39275, saving model to car_model.keras
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.7986 - loss: 0.4684 - val_accuracy: 0.8462 - val_loss: 0.3927
Epoch 3/15
17/26 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8639 - loss: 0.3554
Epoch 3: val_loss improved from 0.39275 to 0.36916, saving model to car_model.keras
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8572 - loss: 0.3646 - val_accuracy: 0.8416 - val_loss: 0.3692
Epoch 4/15
20/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8483 - loss: 0.3582
Epoch 4: val_loss improved from 0.36916 to 0.36368, saving model to 

In [19]:
loss,accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8289 - loss: 0.3939
Test Accuracy: 0.8370


In [20]:
model.predict(X_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


array([[0.03096086],
       [0.5128161 ],
       [0.9455449 ],
       [0.96034825],
       [0.04241194],
       [0.83592606],
       [0.7367151 ],
       [0.14119273],
       [0.6060848 ],
       [0.9533355 ],
       [0.6633308 ],
       [0.23237687],
       [0.6391096 ],
       [0.04851496],
       [0.91761523],
       [0.31219846],
       [0.09007058],
       [0.832127  ],
       [0.8033775 ],
       [0.30277738],
       [0.7215892 ],
       [0.7765088 ],
       [0.05545627],
       [0.7741261 ],
       [0.90714556],
       [0.94539565],
       [0.04187576],
       [0.7942014 ],
       [0.04158965],
       [0.02232729],
       [0.7632656 ],
       [0.10057653],
       [0.71406406],
       [0.94646895],
       [0.9364306 ],
       [0.7800562 ],
       [0.9416167 ],
       [0.05244511],
       [0.93124914],
       [0.65810907],
       [0.8852414 ],
       [0.798761  ],
       [0.66159666],
       [0.1175727 ],
       [0.16126001],
       [0.83186424],
       [0.6695955 ],
       [0.950

In [21]:
checkpoint = ModelCheckpoint(
    'car_model.keras',
    monitor = 'val_precision',
    save_best_only=True,
    verbose=1
)

In [22]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │           1,344 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,373 (40.52 KB)

 Trainable params: 3,457 (13.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,916 (27.02 KB)

In [23]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['precision'])

In [24]:
history = model.fit(X_train, y_train, batch_size=20, epochs=15, validation_split=0.3, callbacks=[early_stop, checkpoint], verbose=1)

Epoch 1/15
19/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3394 - precision: 0.8713
Epoch 1: val_precision improved from inf to 0.82927, saving model to car_model.keras
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 50ms/step - loss: 0.3398 - precision: 0.8722 - val_loss: 0.3591 - val_precision: 0.8293
Epoch 2/15
22/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3231 - precision: 0.8686
Epoch 2: val_precision improved from 0.82927 to 0.82540, saving model to car_model.keras
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.3221 - precision: 0.8705 - val_loss: 0.3659 - val_precision: 0.8254
Epoch 3/15
17/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3321 - precision: 0.8792
Epoch 3: val_precision improved from 0.82540 to 0.81890, saving model to car_model.keras
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.3300 - precision: 0.8822 - val_loss: 0.3618 - val_precision: 0.8189
Epoch 4/15
19/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3089 - precision: 0.8632 
Epoch 4: val_precision did not improve fro

In [25]:
loss,precision = model.evaluate(X_test, y_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.3896 - precision: 0.8386


In [26]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['recall'])

In [50]:
history = model.fit(X_train, y_train, batch_size=20, epochs=15, validation_split=0.3, callbacks=[early_stop, checkpoint], verbose=1)

Epoch 1/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - loss: 0.3410 - recall: 0.9018 - val_loss: 0.3613 - val_recall: 0.8966
Epoch 2/15
 1/26 ━━━━━━━━━━━━━━━━━━━━ 3s 135ms/step - loss: 0.4840 - recall: 0.8182

C:\Users\sania\anaconda3\Lib\site-packages\keras\src\callbacks\model_checkpoint.py:302: UserWarning: Can save best model only with val_precision available.
  if self._should_save_model(epoch, batch, logs, filepath):


26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.3474 - recall: 0.8922 - val_loss: 0.3618 - val_recall: 0.9052
Epoch 3/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.3214 - recall: 0.8733 - val_loss: 0.3639 - val_recall: 0.8966
Epoch 4/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.3024 - recall: 0.8794 - val_loss: 0.3654 - val_recall: 0.8966
Epoch 5/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.3139 - recall: 0.9003 - val_loss: 0.3674 - val_recall: 0.9138
Epoch 6/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.2732 - recall: 0.9169 - val_loss: 0.3625 - val_recall: 0.9052
Epoch 7/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.3376 - recall: 0.8976 - val_loss: 0.3635 - val_recall: 0.9052
Epoch 8/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.3209 - recall: 0.8986 - val_loss: 0.3662 - val_recall: 0.9052
Epoch 8: early stopping
Restoring model weights from the end of the best epoch: 1.


In [54]:
loss,recall = model.evaluate(X_test,y_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.3792 - recall: 0.8413
